# EE 596 Mini Project 1 Q2
Blake Downey and Ryan Maroney

##Question 2
This one is on live paper trading. Take one of your best strategies from Question 1 and run it for 7-10 days (run it live) on the 5 stocks you picked. You can use the buy/sell order methods mentioned in starter code as an example to place orders for live paper trading. So your algorithm is automatically trading live but only on paper. Make sure, the settings for your algorithm enable you to buy and sell within the week (i.e. set your profit margin lower for sell). What's the profit you made?

In [ ]:
%%capture
!pip install alpaca_trade_api
!pip install tensorflow

In [ ]:
import time 
from datetime import datetime, date, timedelta
import alpaca_trade_api as alpaca
from alpaca_trade_api.entity_v2 import BarsV2
#from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import RidgeClassifier as LR_ridge
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import f1_score as f1
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as L

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [ ]:
ENDPOINT="https://paper-api.alpaca.markets"
API_KEY_ID="PKPZ2S22JEES07HVJWPE" # Put in yours here - Needed for paper trading
SECRET_KEY="WEXfJc7ohAd35AWHIKTMsu9RgKRqCpIbHVwWIvA5" # Put in yours here - Needed for paper trading

'''
Base Trading strategy class to be shared among strategies
'''
class TradingStrategy:
    def __init__(self, STOCK, DAYS):
        self.api = alpaca.REST(API_KEY_ID, SECRET_KEY, ENDPOINT)
        self.STOCK = STOCK
        self.SELL_LIMIT_FACTOR = 1.01 # 1 percent margin
        
        # Anything else you want to initialize or a method you want to
        # call during initialization of class - Feel free to add
        self.DAYS = DAYS
        self.get_bars()
        self.train_bars = None

        self.BACKTEST_START_MONEY = 10000.00
        self.backtest_money = self.BACKTEST_START_MONEY
        self.action_state = None
        self.actions = []
        self.current_num_stocks = 0
        self.current_stock_amount = 0.00

        # get the months and days of the stocks
        self.generate_date_stamps()

        self.hardcode_dates()
        
        # Get past 90 days closing prices
        #self.get_past90_closing_prices()

    def hardcode_dates(self):
      self.sell_earning_dates = {
          'AMZN' : (date(2022,2,2), date(2022,2,3)),
          'GOOGL' : (date(2022,1,31), date(2022,2,1)),
          'AAPL' : (date(2022,4,25), date(2022,4,26)),
          'NVDA' : (date(2022,5,23), date(2022,5,24)),
          'AMD' : (date(2022,2,1), date(2022,2,2))
      }

    def generate_date_stamps(self):
      dates_temp = self.bars['close'].index
      self.dates = []
      for i in range(len(dates_temp)):
        self.dates.append(dates_temp[i].date())

    def get_bars(self):
        self.bars = self.api.get_bars(self.STOCK, alpaca.TimeFrame.Day, date.today() - timedelta(days=self.DAYS), date.today() - timedelta(days=1), adjustment='raw').df
    
    def get_train_bars(self, percent_train=0.75):
        self.num_train_bars = int(len(self.bars)*percent_train)
        self.train_bars = self.bars.iloc[:self.num_train_bars,:]
        self.future_bars = self.bars.iloc[self.num_train_bars:,:]

    def get_past90_closing_prices(self):  
        barset = self.api.get_barset(self.STOCK, 'day', limit=90)
        bars = barset[self.STOCK]
        self.past90_closing_prices = [bars[index].c for index in range(len(bars))]
    
    def get_current_price(self):
        return float(self.api.get_last_trade(self.STOCK).price)
    
    def get_quantity_buy(self):
        if int(float(self.api.get_account().cash)) > 0:
            return int((float(self.api.get_account().cash)/2) \
                       /self.get_current_price())
        else:
            return 0

    def get_sell_quantity(self):
        positions=self.api.list_positions()
        for position in positions:
            if position.symbol==self.STOCK:
                return int(position.qty)
        return 0
        
    def exists_buy_order(self):
        # Identifies if a buy order exists for a stock
        orders = self.api.list_orders()
        for order in orders:
            if order.side=="buy" and order.symbol==self.STOCK:
                return True
        
        return False
    
    def have_bought_stock(self):
        positions=self.api.list_positions()
        for position in positions:
            if position.symbol==self.STOCK and int(position.qty)==self.NEW_QUANTITY + self.EXISTING_QUANTITY:
                return True
        return False
        
        
    def get_buy_price(self):
        # Identify the buying price for a stock
        positions=self.api.list_positions()
        for position in positions:
            if position.symbol==self.STOCK:
                return float(position.cost_basis)/int(position.qty)
    
    
    def buy_market_order(self):
        # Buy the stock at market price (This is for paper-trading)
        if self.NEW_QUANTITY > 0:
            self.api.submit_order(self.STOCK, \
                        qty=self.NEW_QUANTITY,\
                        side="buy",\
                        type="market", \
                        time_in_force="day",
                        order_class=None)
        
    def buy_limit_order(self,base_price):
        if self.NEW_QUANTITY > 0:
            self.api.submit_order(self.STOCK, \
                        qty=self.NEW_QUANTITY,\
                        side="buy",\
                        type="limit", \
                        limit_price=base_price, \
                        time_in_force="day",
                        order_class=None)
        
    def sell_market_order(self):
        if self.get_sell_quantity() > 0: # if there are more than 0 qty 
            self.api.submit_order(self.STOCK, \
                        qty=self.get_sell_quantity(),\
                        side="sell",\
                        type="market",\
                        time_in_force="day",
                        order_class=None)

    def sell_limit_order(self):
        if self.get_sell_quantity() > 0:
            self.api.submit_order(self.STOCK, \
                        qty=self.get_sell_quantity(),\
                        side="sell",\
                        type="limit", \
                        limit_price=self.SELL_LIMIT_PRICE, \
                        time_in_force="day",
                        order_class=None)
        
    def identify_strategy_for_selling(self):
        # If you have multiple strategies
        # Pick between them here - Or use ML to help identify 
        # your strategy
        pass
        
    def market_buy_strategy(self):
        # Providing a simple trading strategy here:
        # Buy at market price if conditions are favorable for buying
        # Sell at a limit price that is determined based on buying price
        # This strategy doesn't use any ML here - You may want to use
        # appropriate libraries to train models + use the trained strategy 
        # here
        
        # Get existing quantity
        positions = self.api.list_positions()
        self.EXISTING_QUANTITY = 0
        for position in positions:
            if position.symbol == self.STOCK:
                self.EXISTING_QUANTITY += int(position.qty)
                
        # MARKET BUY order
        self.NEW_QUANTITY=self.get_quantity_buy()
        
        if self.NEW_QUANTITY == 0:
            return "ZERO EQUITY"
        
        if not self.exists_buy_order():
            self.buy_market_order()
            
        
        # BRACKET SELL order
        # Initiate sell order if stock has been bought
        # If not, wait for it to be bought
        while not self.have_bought_stock():
            #print(self.api.positions)
            #print(self.NEW_QUANTITY + self.EXISTING_QUANTITY)
            time.sleep(1)
        
        if self.have_bought_stock():
            buy_price=self.get_buy_price()
            self.SELL_LIMIT_PRICE=int(float(buy_price))*self.SELL_LIMIT_FACTOR
            
            # Initiate Sell order
            self.sell_limit_order()
      
    def your_best_strategy(self):
        # Implement here or add other methods to do the same
        pass
       
    '''
    ''' 
    def preprocess_strategy(self):
      # Implement in child class
      pass

    '''
    '''
    def execute_strategy(self, current_price):
      # Implement in child class
      pass

    '''
    Backtest evaluation, buy max of 1 stock, sell all stocks on sell
    '''
    def execute_backtest_trade(self, action, price):
      if action == "BUY":
        if self.current_num_stocks < 1:
          self.action_state = "BUY"
          self.backtest_money -= price
          self.current_num_stocks += 1

      elif action == "SELL":
        if self.current_num_stocks > 0:
          self.action_state = "SELL"
          self.backtest_money += self.current_num_stocks * price
          self.current_num_stocks = 0

      else: # hold current position
        pass

    '''
    execute paper trade
    '''
    def execute_trade(self, action):
      if action == "BUY":
        self.NEW_QUANTITY = self.get_quantity_buy()
        
        if self.NEW_QUANTITY == 0:
          print("ZERO EQUITY")
        else:
          self.buy_market_order()

      elif action == "SELL":
        if self.have_bought_stock():
          print("sell")
          buy_price=self.get_buy_price()
          self.SELL_LIMIT_PRICE=int(float(buy_price))*self.SELL_LIMIT_FACTOR
          
          # Initiate Sell order
          self.sell_limit_order()

      else: # hold current position
        pass

    '''
    Use bars to execute strategy rather than list of prices
    '''
    def run_backtest_bars(self, percent_train=0.75):
      self.backtest_money = self.BACKTEST_START_MONEY

      self.actions = []

      self.get_train_bars(percent_train)
      self.preprocess_strategy(self.train_bars)

      for index, future_bar in self.future_bars.iterrows():
        action = self.execute_strategy(self.train_bars, future_bar['close'], is_backtest=True)

        # Sell before earnings dates
        if index == self.sell_earning_dates[self.STOCK][0]:
          action = 'SELL'

        self.actions.append(action)
        self.train_bars = self.train_bars.append(future_bar)
        self.execute_backtest_trade(action, future_bar['close'])

      self.execute_backtest_trade("SELL", self.future_bars.iloc[-1]['close'])

      return self.actions, self.backtest_money
        

    '''
    run backtest with list of prices
    '''
    def run_backtest(self, percent_train=0.4):
      prices = self.bars['close'].values
      train_days = int(len(prices)*(percent_train))
      train_prices = prices[:train_days]
      future_prices = prices[train_days:]

      
      dates = self.bars['close'].index
      future_dates = dates[train_days:]

      self.backtest_money = self.BACKTEST_START_MONEY
      self.actions = []

      self.preprocess_strategy(train_prices)

      for i,price in enumerate(future_prices):
        action = self.execute_strategy(price)

        if future_dates[i] == self.sell_earning_dates[self.STOCK][0]:
          action = 'SELL'

        self.actions.append(action)
        self.execute_backtest_trade(action, price)

      self.execute_backtest_trade("SELL", future_prices[-1])

      return self.actions, self.backtest_money

    '''
    plot backtest results
    '''
    def plot_backtest_trades(self):
      prices = self.bars['close'].tail(len(self.actions)).values

      plt.figure(figsize=(10,10))
      plt.title(f"Backtest Trading {self.STOCK} (Percent Profits: {100.0*(self.backtest_money - self.BACKTEST_START_MONEY) / self.BACKTEST_START_MONEY})")
      plt.plot(prices)

      for i, action in enumerate(self.actions):
        if action == "BUY":
          plt.plot(i, prices[i], marker='^', markersize=15, markeredgecolor="green", markerfacecolor="green")
        elif action == "SELL":
          plt.plot(i, prices[i], marker='v', markersize=15, markeredgecolor="red", markerfacecolor="red")

      plt.show()

    '''
    run live trading
    '''
    def run_live_paper_strategy(self):
      
      self.preprocess_strategy() # Maybe not needed?
      current_price = self.get_current_price()
      action = self.execute_strategy(current_price)
      print(current_price, action)
      self.execute_trade(action)
 

In [ ]:
'''
Base moving average class
'''
class MovingAverage:
  '''
  '''
  def __init__(self, values):
    self.mean = 0

  '''
  '''
  def calculate_average(self, values):
    pass

  '''
  '''
  def update_average(self, new_data_point):
    pass

'''
Simple moving average taking average of last n days
'''
class SimpleMovingAverage(MovingAverage):
  '''
  '''
  def __init__(self, values, period):
    self.period = period
    super().__init__(values)
    self.calculate_average(values)

  '''
  '''
  def calculate_average(self, values):
    self.values = values[len(values) - self.period:]
    self.mean = self.values.mean()

  '''
  '''
  def update_average(self, new_data_point):
    self.values = np.append(self.values, new_data_point)[1:]
    self.mean = self.values.mean()

'''
Exponential moving average weight most recent day base on beta value
'''
class ExponentialMovingAverage(MovingAverage):
  '''
  '''
  def __init__(self, values, beta=0.3):
    super().__init__(values)
    self.beta = beta
    self.calculate_average(values)

  '''
  '''
  def calculate_average(self, values):
    self.mean = 0

    for i in range(len(values)):
      self.mean += (values[i] + self.mean) * ((1-self.beta)**(len(values) - 1 - i))
      self.mean *= self.beta

  '''
  '''
  def update_average(self, new_data_point):
    self.mean *= 1 - self.beta
    self.mean += self.beta * new_data_point

'''
Baseline crossover strategy comparing when two moving averages crossover
'''
class BaselineCrossoverStrategy(TradingStrategy):
  '''
  '''
  def __init__(self, STOCK, DAYS, fast_method="SMA", slow_method="SMA", sma_fast_period=5, sma_slow_period=20, ema_fast_beta=0.5, ema_slow_beta=0.5):
    super().__init__(STOCK, DAYS)

    if fast_method == "SMA":
      self.fast_method = SimpleMovingAverage(self.bars['close'].values, sma_fast_period)
    elif fast_method == "EMA":
      self.fast_method = ExponentialMovingAverage(self.bars['close'].values, ema_fast_beta)

    if slow_method == "SMA":
      self.slow_method = SimpleMovingAverage(self.bars['close'].values, sma_slow_period)
    elif slow_method == "EMA":
      self.slow_method = ExponentialMovingAverage(self.bars['close'].values, ema_slow_beta)

    self.crossover_state = self.get_crossover_state()

  '''
  '''
  def get_crossover_state(self):
    return self.fast_method.mean > self.slow_method.mean
      
  '''
  '''
  def preprocess_strategy(self, data=None):
    if not isinstance(data, type(None)):
      self.fast_method.calculate_average(data)
      self.slow_method.calculate_average(data)

    self.crossover_state = self.get_crossover_state()

  '''
  '''
  def execute_strategy(self, current_price):
    action = None

    self.fast_method.update_average(current_price)
    self.slow_method.update_average(current_price)

    new_state = self.get_crossover_state()

    if self.crossover_state and not new_state:
      action = "SELL"
    elif not self.crossover_state and new_state:
      action = "BUY"

    self.crossover_state = new_state

    return action

  def generate_fast_slow(self):
    slow, fast = [], []

    for i, price in enumerate(self.bars['close'].values):

        self.fast_method.update_average(price)
        self.slow_method.update_average(price)
        fast.append(self.fast_method.mean)
        slow.append(self.slow_method.mean)

    return fast, slow

In [ ]:
# ---------------------------------------------------------------------------------#

#                         Definitions for Generating Data Frames                           #

# ---------------------------------------------------------------------------------#

'''
Single Data Points information per data point, ie 7 features plus label
'''
def create_data(yrs=1, STOCK='GOOG', fast_method='EMA', slow_method='SMA',
                            fast_sma=5, slow_sma=15, fast_ema=0.5, slow_ema=0.2, gainz_threshold = 1.01, debug=False) :
    if debug:                         
      print('----------------------------')
      print(f'Gathering Data for {STOCK}')
      print('----------------------------')
      
    # Create instance of BCS
   
    ML_strat = BaselineCrossoverStrategy(
        STOCK, 232*yrs, 
        fast_method=fast_method,
        slow_method=slow_method, 
        sma_fast_period=fast_sma, sma_slow_period=slow_sma, 
        ema_fast_beta=fast_ema, ema_slow_beta=slow_ema
        )
      
    # set the sell limit factor for determining when to sell
    ML_strat.SELL_LIMIT_FACTOR = gainz_threshold

    # Closing Prices Data
    closing_prices = ML_strat.bars['close'].values

    # Get Fast and Slow SMA or EMA curves
    fast, slow = ML_strat.generate_fast_slow()

    # Set sma_var to correct variable
    sma_var = []
    if fast_method == 'SMA': sma_var[:] = slow[:]
    elif slow_method == 'SMA': sma_var[:] = fast[:]
    else: sma_var[:] = slow[:]

    # Get STD, upper std and lower std
    upper_dev, lower_dev, devs = [], [], []
    for i in range(len(closing_prices)):
        devs.append(abs(closing_prices[i] - sma_var[i])) # use whichever is the SMA: slow here
        upper_dev.append(slow[i] + 2*devs[i]) 
        lower_dev.append(slow[i] - 2*devs[i]) 

    # Graph Data
    if debug:
      x = np.arange(1, len(closing_prices)+1 )
      y = closing_prices
      plt.plot(x,y) # plot the actual prices
      plt.plot(slow) # plot the slow method
      plt.plot(upper_dev) # plot upper standard devs
      plt.plot(lower_dev) # plot lower standard devs
      plt.plot(fast) # plot the fast method
    
    
    # Generate Slope Points from slow SMA curve
    slopes = [0]
    last_slow_point = slow[0]
    for i in range(1,len(slow)):
        curr_point = slow[i]
        slopes.append(curr_point - last_slow_point)
        last_slow_point = curr_point

    #print(len(data))
    #print(slopes)

     # Create Pandas DataFrame
    df = pd.DataFrame(index=[np.arange(len(closing_prices))])
    df['Closing Prices'] = closing_prices
    df['SMA'] = slow
    df['EMA'] = fast
    df['devs'] = devs
    df['upper_dev'] = upper_dev
    df['lower_dev'] = lower_dev
    df['slopes'] = slopes

    #print(df.head(-1))
    for i in range(15):
        df = df.drop(index=i)
    #print(df.shape)

    #Generate Labels
    labels = [] # 0 means hold, 1 means buy, 2 means sell!
    bought = False
    cropped_data = df['Closing Prices'].values
    cropped_slopes = df['slopes'].values
    buy_price = 0

    for i in range(len(cropped_data)-1):
        if cropped_data[i+1] > cropped_data[i] and cropped_data[i-2] > cropped_data[i] and cropped_data[i-1] > cropped_data[i]: # cropped_data[i+2] > cropped_data[i] and 
            labels.append(1)
            bought = True
            buy_price = cropped_data[i]
        elif cropped_data[i+1] < cropped_data[i] and cropped_data[i-2] < cropped_data[i] and cropped_data[i-1] < cropped_data[i]: # cropped_data[i+2] < cropped_data[i] and 
            labels.append(2)
            bought = False
        else: 
            labels.append(0)
    #drop the last index since it doesnt have a label
    #df = df.drop(index=len(closing_prices)-2)
    #df = df.drop(index=len(closing_prices)-3)
    #df = df.drop(index=len(closing_prices)-4)
    #add labels to df
    labels.append(0)
    df['labels'] = labels
    
    return df

'''
Create a mega dataframe of appended single stock dataframes
'''
def concat_dfs(STOCKS, yrs, fast_method='EMA', slow_method='SMA', fast_sma=5, slow_sma=15, 
               fast_ema=0.5, slow_ema=0.2,gainz_threshold=1.01, stack=False, debug=False):
    # create mega_frame from first stock df
    mega_frame = create_data(yrs=yrs, STOCK=STOCKS[0], fast_method=fast_method, slow_method=slow_method,
                             fast_sma=fast_sma, slow_sma=slow_sma, fast_ema=fast_ema, slow_ema=slow_ema)
    if stack: mega_frame = feature_stack(mega_frame)
    if debug: print(f'Shape of DataFrame: {mega_frame.shape}')
    for i in range(1,len(STOCKS)):
        temp_df = create_data(yrs=yrs, STOCK=STOCKS[i], fast_method=fast_method, slow_method=slow_method,
                              fast_sma=fast_sma, slow_sma=slow_sma, fast_ema=fast_ema, slow_ema=slow_ema)
        if stack: temp_def = feature_stack(temp_df)
        mega_frame = pd.concat([mega_frame, temp_df])
        if debug: print(f'Shape of DataFrame: {mega_frame.shape}')
    return mega_frame


'''
Create DataFrame from existing dataframe where each datapoint has closing price,
SMA, and EMA from previous 4 datapoints. 
Input Dimensions (n,8) -- 7 + label
Output Dimensions (n-4,24) 23 + label
'''
def feature_stack(df1):
    #define the extra columns
    extra_cols = ['CP', 'SMA', 'EMA', 'slope','CP', 'SMA', 'EMA', 'slope','CP', 'SMA',
    'EMA', 'slope','CP', 'SMA', 'EMA', 'slope']

    # grab the prices for continued modification 
    cps = df1['Closing Prices'].values 
    sma = df1['SMA'].values
    ema = df1['EMA'].values
    slope = df1['slopes'].values

    count = 0

    for i in range(len(extra_cols)):
        if i%4 == 0: count += 1  # iterator for column labels
        if extra_cols[i] == 'CP': # if the value is Closing Price
            cps = np.delete(cps,-1) # remove the last element 
            cps = np.insert(cps,0,0) # make the first element 0 
            df1[extra_cols[i]+str(count)] = cps # append that column to the correct column in the df
        elif extra_cols[i] == 'SMA':
            sma = np.delete(sma,-1)
            sma = np.insert(sma,0,0)
            df1[extra_cols[i]+str(count)] = sma
        elif extra_cols[i] == 'EMA':
            ema = np.delete(ema,-1)
            ema = np.insert(ema,0,0)
            df1[extra_cols[i]+str(count)] = ema
        elif extra_cols[i] == 'slope':
            slope = np.delete(slope,-1)
            slope = np.insert(slope,0,0)
            df1[extra_cols[i]+str(count)] = slope

    for i in range(15,19):
        df1 = df1.drop(index=i)

    return df1



In [ ]:
# ---------------------------------------------------------------------------------#

#                               Labeled Data Example Graph                                      #

# ---------------------------------------------------------------------------------#
def labeled_data_ex(stock='GOOG', yrs=(5/12)):

    df1 = create_data(yrs=yrs, STOCK=stock)
    df1_target = df1.pop('labels')

    actual_data = df1['Closing Prices']
    indices = actual_data.index
    print(f'Actual Labeled Data Example Graphed for {stock} for {yrs} years')
    plt.figure(figsize=(20,10))
    plt.plot(df1['Closing Prices'].values)

    for i,label in enumerate(df1_target):
        x = indices[i][0] - 15
        if label == 1:
            plt.plot(x, actual_data.values[i], marker='^', markersize=10, markeredgecolor="green", markerfacecolor="green")
        elif label == 2:
            plt.plot(x, actual_data.values[i], marker='v', markersize=10, markeredgecolor="red", markerfacecolor="red")
    plt.title(f'Algorithmic Labels for {stock} stock', color='white', size=25)    
    plt.show()

In [ ]:
# ---------------------------------------------------------------------------------#

#                          Definition for ML Ridge Classifier                                   #

# ---------------------------------------------------------------------------------#
def ML(X_train, y_train, X_test=None, y_test=None, max_iter=800, alpha=0.01, normalize=True, debug=False):
    if normalize:
        mu, std = X_train.mean(), X_train.std()
        X_train = (X_train-mu) / std

        if debug:
          mu, std = X_test.mean(), X_test.std()
          X_test = (X_test-mu) / std

    model_ridge = LR_ridge(max_iter=max_iter, alpha=alpha).fit(X_train, y_train)

    if debug:
      '''
      debug mode: printing results for predictions and plotting results. 
      '''
      pred_train = model_ridge.predict(X_train)
      pred_test = model_ridge.predict(X_test)

      print(f'\nf1 score train: {f1(y_train, pred_train, average="weighted")}')
      print(f'f1 score test: {f1(y_test, pred_test, average="weighted")}')

      print(f'\nDimensions of Train Set: {X_train.shape}')
      print(f'Dimensions of Test Set: {X_test.shape}')

      '''
      The ML algorithm predictions with current parameters and labeled train data
      '''
      actual_data = X_test['Closing Prices']
      indices = actual_data.index

      print(f'Logistic Regression Predictions Graphed')
      plt.figure(figsize=(20,10))
      plt.plot(X_test['Closing Prices'].values)

      for i,pred in enumerate(pred_test):
          x = indices[i][0] - indices[0][0]
          if pred == 1:
              plt.plot(x, actual_data.values[i], marker='^', markersize=10, markeredgecolor="green", markerfacecolor="green")
          elif pred == 2:
              plt.plot(x, actual_data.values[i], marker='v', markersize=10, markeredgecolor="red", markerfacecolor="red")
      plt.title('Logistic Regression Predicted Values', color='white', size=25)

      plt.show()

    return model_ridge


# ---------------------------------------------------------------------------------#

#                        Definition for Generating ML Model                                  #

# ---------------------------------------------------------------------------------#
def generate_ML_model(stock, debug=False, alpha=0.001):
    # Hyper Params
    STOCKS = ['GOOG', 'MSFT','FB'] 
    test_stock = stock
    yrs = 1
    fast_sma = 5
    slow_sma = 15 # active
    fast_ema = 0.6 # active
    slow_ema = 0.2
    fast_method = 'EMA'
    slow_method = 'SMA'
    stack = True

    if debug: 
      # What the Labeled Data looks like for given stock:
      labeled_data_ex(test_stock)

    # Generate Multi Stock Train Dataset w/out feature stacking for ML - Ridge Regression
      print('Generating Train DataFrame:')
    X_train = concat_dfs(STOCKS, yrs, fast_method=fast_method, slow_method=slow_method, 
                    fast_sma=fast_sma, slow_sma=slow_sma, fast_ema=fast_ema, slow_ema=slow_ema,
                    stack=stack, debug=debug)

    # Save df to csv
    if debug:
      X_train.to_csv('./data.csv')
    
    # use labels column as target (y_train)
    y_train = X_train.pop('labels')

    # Generate Test Data
    if debug:
      print('\nGenerating Test DataFrame:')
      X_test = create_data(yrs=(5/12), STOCK=test_stock, fast_method=fast_method, slow_method=slow_method)
      if stack: X_test = feature_stack(X_test)
      y_test = X_test.pop('labels')
      
      # Run the ML Ridge Regression on the Data
      print(f'\nHyper-Parameter Settings:')
      print(f' years of data for train/val: {yrs}\n \
      Stocks for Train: {STOCKS}\n \
      Stock for Test: \'{test_stock}\'\n \
      Fast Method = {fast_method}\n \
      Slow Method = {slow_method}\n \
      Fast EMA Beta: {fast_ema}\n \
      Slow EMA Beta: {slow_ema}\n \
      Fast SMA Value: {fast_sma}\n \
      Slow SMA Value: {slow_sma}\n \
      Feature Stacking: {stack}\n')
      print('\nML Ridge Classifier')
        
    if debug: model = ML(X_train, y_train, X_test, y_test, alpha=alpha, debug=debug)
    else: model = ML(X_train, y_train, alpha=alpha)

    return model


# ---------------------------------------------------------------------------------#

#                   Definition for ML Based Buy/Sell Strategy                            #

# ---------------------------------------------------------------------------------#
class ML_Strategy(TradingStrategy):
 
  def __init__(self, STOCK, DAYS, MODEL, debug=False):
    super().__init__(STOCK, DAYS)
    self.model = MODEL
    self.available_cash = self.BACKTEST_START_MONEY
    self.sum_price = 0
    self.count_buys = 0
    self.debug = debug
    self.actions = []

    # needed parameters for creating backtesting data and live trade previous data
    # MUST BE THE SAME PARAMETERS USED IN TRAINING OF ML MODEL - static
    self.yrs = (5/12)
    self.fast_sma = 5
    self.slow_sma = 15 # active
    self.fast_ema = 0.6 # active
    self.slow_ema = 0.2
    self.fast_method = 'EMA'
    self.slow_method = 'SMA'

    self.SELL_LIMIT_FACTOR  = 1.01

  '''
  live paper trading
  '''
  def live_trade_setup(self):
    self.data = feature_stack(create_data(yrs=(3/12), STOCK=self.STOCK, fast_method=self.fast_method, slow_method=self.slow_method))

  def get_q_buy(self):
    if int(float(self.api.get_account().cash)) > 0:
        return int((float(self.api.get_account().cash)/5) / self.get_current_price())
    else:
        return 0

  def get_limited_q_buy(self):
    if int(float(self.api.get_account().cash)) > 0:
        return int((float(self.api.get_account().cash)/10) / self.get_current_price())
    else:
        return 0

  '''
  execute paper trade with action
  '''
  def execute_paper_trade(self, action):
    if action == 1:
      self.NEW_QUANTITY = self.get_q_buy()
      
      if self.NEW_QUANTITY == 0:
        print("ZERO EQUITY")
      else:
        self.buy_market_order()

    elif action == 2:
      positions = self.api.list_positions()
      flag = False
      for pos in positions:
        if pos.symbol == self.STOCK:
          flag = True
      if flag:
        print("sell")
        buy_price=self.get_buy_price()
        self.SELL_LIMIT_PRICE=int(float(buy_price))*self.SELL_LIMIT_FACTOR
      else: print('No shares of this stock owned...')
        
      # Initiate Sell order
      self.sell_market_order()

    else: # hold current position
      pass
    '''
      positions = self.api.list_positions()
      flag = False
      for pos in positions:
        if pos.symbol == self.STOCK:
          flag = True
      if not flag:
        print('Recommended Hold -- there exist no positions for this stock. Buying a limited number of shares to start trading...')
        self.NEW_QUANTITY = self.get_limited_q_buy()
        if self.NEW_QUANTITY == 0:
          print("ZERO EQUITY")
        else:
          print(f'Stock purchased: {self.NEW_QUANTITY}\n')
          self.buy_market_order()
    '''
  '''
  Generate data using previous stock data
  '''
  def generate_backtest_data(self):
    data = create_data(yrs=self.yrs, STOCK=self.STOCK, fast_method=self.fast_method, slow_method=self.slow_method)
    data = feature_stack(data)
    _ = data.pop('labels') # remove the labels from the data

    self.data = data
  
  '''
  Helper functions for backtesting / possibly live trading
  '''
  def get_buy_quantity_backtest(self):
    if self.count_buys < 4:
      return int((self.available_cash / ((1/(self.count_buys+1)) * 7))) / self.current_stock_amount
    else: 
      return 0

  def reset_vars(self):
    #reset the counts upon selling -- ie sells all stock
    self.sum_price = 0
    self.current_num_stocks = 0
    self.count_buys = 0

  def sell_backtest(self, force=False):
    if force: 
      self.available_cash += self.current_stock_amount * self.current_num_stocks
      self.actions.append(2)
      if self.debug:
        print(f'Sell: {self.current_num_stocks} @ {self.current_stock_amount}')
        print(f'Available Cash: {self.available_cash}\n')
      self.reset_vars()
      return

    # only sell if greater than threshold sell limit factor * average price of the stocks bought    
    if (self.current_num_stocks * self.current_stock_amount) >= self.SELL_LIMIT_FACTOR * self.sum_price and self.current_num_stocks > 0:
      self.available_cash += self.current_stock_amount * self.current_num_stocks
      self.actions.append(2)
      if self.debug:
        print(f'Sell: {self.current_num_stocks} @ {self.current_stock_amount}')
        print(f'Available Cash: {self.available_cash}\n')
      self.reset_vars()
      return 
    
    self.actions.append(0) # if stock doesnt sell - append a 0 for hold

  def buy_backtest(self):
    # simulate buy by tracking quantity bought and available cash
    q = self.get_buy_quantity_backtest()
    if q > 0: 
      self.count_buys += 1
      self.actions.append(1)
    else: self.actions.append(0)
    self.available_cash -= q * self.current_stock_amount
    self.current_num_stocks += q
    if self.debug:
      print(f'Buy: {q} @ {self.current_stock_amount} = {q*self.current_stock_amount}')
      print(f'Available Cash: {self.available_cash}')

    # increment total sum 
    self.sum_price += self.current_stock_amount * q
    if self.debug:
      print(f'sum price: {self.sum_price}\n')

    if self.current_num_stocks == 0: self.reset_vars()


  def plot(self):
    actual_data = self.data['Closing Prices']
    indices = actual_data.index
    print(f'Logistic Regression Buy/Sell Actions Taken')
    plt.figure(figsize=(20,10))
    plt.plot(self.data['Closing Prices'].values)

    if self.debug:
      print(f'num actions: {len(self.actions)}')
      print(f'num indices: {len(indices)}')

    for i,action in enumerate(self.preds):
        if i >= len(indices): break
        x = indices[i][0] - indices[0][0]
        if action == 1:
            plt.plot(x, actual_data.values[i], marker='^', markersize=10, markeredgecolor="green", markerfacecolor="green")
        elif action == 2:
            plt.plot(x, actual_data.values[i], marker='v', markersize=10, markeredgecolor="red", markerfacecolor="red")
        #else:
         #   plt.plot(x, actual_data.values[i], marker='o', markersize=10, markeredgecolor="yellow", markerfacecolor="yellow")
    plt.show()

  
  '''
  Implement Backtesting
  '''
  def run_backtest(self):  
    # generate the data needed for backtesting
    self.generate_backtest_data() # data stored in self.data

    self.preds = []

    if self.debug:
      print(self.dates)

    offset = 19
    
    # loop through data to do day by day predictions 
    for i in range(self.data.shape[0]):
      # if day before earnings -- force sell
      if self.dates[i+offset] == self.sell_earning_dates[self.STOCK][0] and self.current_num_stocks > 0:
        if self.debug: print(f'{self.dates[i+offset]} Day Before Earnings... Force Sell')
        self.sell_backtest(force=True)
      # if day of earnings -- hold else execute code below
      elif not self.dates[i+offset] == self.sell_earning_dates[self.STOCK][1]:
        
        # get price at EOD
        self.current_stock_amount = self.data.iloc[i]['Closing Prices']
        
        #predict for that data point
        pred_action = self.model.predict(self.data.iloc[i].values.reshape(1,-1))

        self.preds.append(pred_action)
        
        if self.debug: print(f'date: {self.dates[i+offset]}')

        #determine course of action
        if pred_action == 1: # buy
          if self.debug:
            print('------buy------')
          self.buy_backtest()
        elif pred_action == 2: # sell
          if self.debug:
            print('------sell------')
          self.sell_backtest()
        else: # hold
          if (self.current_num_stocks * self.current_stock_amount) >= self.SELL_LIMIT_FACTOR * self.sum_price and self.current_num_stocks > 0:
            self.sell_backtest()
          else: self.actions.append(0)
    
    if self.current_num_stocks > 0:
      if self.debug:
        print(f'\nNum stocks to fire sale at end: {self.current_num_stocks}')
      self.sell_backtest(force=True)

    self.net_gainz_percent = ((self.available_cash - self.BACKTEST_START_MONEY) / self.BACKTEST_START_MONEY ) * 100
    net_gainz_money = (self.available_cash - self.BACKTEST_START_MONEY)

    self.plot()

    print(f'Starting Cash: {self.BACKTEST_START_MONEY}\nEnding Cash: {self.available_cash : .2f}\nNet Gains: {self.net_gainz_percent : .2f}%\n')

    return self.net_gainz_percent, net_gainz_money

In [ ]:
# ---------------------------------------------------------------------------------#

#                        Execute ML Based Buy/Sell Strategy                               #

# ---------------------------------------------------------------------------------#

stocks = ['AMZN', 'GOOGL', 'AAPL', 'NVDA', 'AMD']
models = []
alphas = [0.001, 0.00009, 0.0009, 0.0009, 0.0009]
debug = False

print('Key:\n\t1 == buy\n\t2 == sell\n\t0 == hold\n')

print('Hardcoding actions to sell for every stock as this is the last day of trading...\n')

for stock in range(len(stocks)):
  models.append(generate_ML_model(stocks[stock],debug,alphas[stock]))
  strat = ML_Strategy(stocks[stock], int(232*5/12), models[stock], debug)

  #strat.run_backtest()

  strat.live_trade_setup()

  # generate new data for today to predict
  new_data = []
  #append current price
  cp = strat.get_current_price()
  new_data.append(cp)
  #print('new CP: ', new_data[0])

  # get new sma value
  end = strat.data.index[-1][0] + 1
  start = end - 14
  sma_val = 0
  for i in range(start, end):
    sma_val += strat.data['Closing Prices'][i]
  sma_val = (sma_val+cp)/(end-start+1)
  #append new sma value
  new_data.append(sma_val)
  #print('new sma val: ', sma_val)

  # get new ema value
  beta = 0.5
  curr_ema = strat.data['EMA'][strat.data.index[-1][0]]
  curr_ema *= 1 - beta
  curr_ema += beta * new_data[0]
  #append new ema value
  new_data.append(curr_ema)
  #print('new ema val: ', curr_ema)

  # get new std, upper and lower std
  dev = abs(cp - curr_ema)
  upper_dev = (sma_val + 2*dev) 
  lower_dev = (sma_val - 2*dev) 
  #append the results
  new_data.append(dev)
  new_data.append(upper_dev)
  new_data.append(lower_dev)
  #print('new dev:', dev)
  #print('new up_dev:',upper_dev)
  #print('new low_dev:',lower_dev)

  # get new point slope
  sma_1 = sma_val
  sma_2 = strat.data['SMA'][strat.data.index[-1][0]]
  slope = sma_1 - sma_2
  # append the slope result
  new_data.append(slope)
  #print('new slope: ', slope)

  # fill out the rest of the values from previous datatable values
  new_data.append(strat.data['Closing Prices'][strat.data.index[-1][0]])
  new_data.append(strat.data['SMA'][strat.data.index[-1][0]])
  new_data.append(strat.data['EMA'][strat.data.index[-1][0]])
  new_data.append(strat.data['slopes'][strat.data.index[-1][0]])

  new_data.append(strat.data['CP1'][strat.data.index[-1][0]])
  new_data.append(strat.data['SMA1'][strat.data.index[-1][0]])
  new_data.append(strat.data['EMA1'][strat.data.index[-1][0]])
  new_data.append(strat.data['slope1'][strat.data.index[-1][0]])

  new_data.append(strat.data['CP2'][strat.data.index[-1][0]])
  new_data.append(strat.data['SMA2'][strat.data.index[-1][0]])
  new_data.append(strat.data['EMA2'][strat.data.index[-1][0]])
  new_data.append(strat.data['slope2'][strat.data.index[-1][0]])

  new_data.append(strat.data['CP3'][strat.data.index[-1][0]])
  new_data.append(strat.data['SMA3'][strat.data.index[-1][0]])
  new_data.append(strat.data['EMA3'][strat.data.index[-1][0]])
  new_data.append(strat.data['slope3'][strat.data.index[-1][0]])

  #print('new data row: \n', new_data)
  #print('len of new data row: \n', len(new_data))

  index = ['Closing Prices', 'SMA', 'EMA',	'devs',	'upper_dev',	'lower_dev',	'slopes',	'CP1',	'SMA1',	'EMA1',	'slope1',	'CP2',	'SMA2',	'EMA2',	'slope2',	'CP3',	'SMA3',	'EMA3',	'slope3',	'CP4',	'SMA4',	'EMA4',	'slope4']
  pd_series = pd.Series(new_data, index=index)
  new_df = strat.data.append(pd_series, ignore_index=True)
  #remove labels
  _ = new_df.pop('labels')
  
  #predict action for today
  action = models[stock].predict(new_df.iloc[new_df.index.values[-1]].values.reshape(1,-1))
  print(f'Stock: {stocks[stock]} \ Action: {action}')

  # code below was used on the last day to force sell out for each  stock
  '''
  new_action = 2
  print(f'Algorithm said {action[0]}, action changed to {new_action} to force sell out.')
  '''

  strat.execute_paper_trade(action)

print('Paper Trading Completed for Today!')

Key:
	1 == buy
	2 == sell
	0 == hold

Hardcoding actions to sell for every stock as this is the last day of trading...

Stock: AMZN \ Action: [1]
Algorithm said 1, action changed to 2 to force sell out.
sell
Stock: GOOGL \ Action: [1]
Algorithm said 1, action changed to 2 to force sell out.
sell
Stock: AAPL \ Action: [1]
Algorithm said 1, action changed to 2 to force sell out.
No shares of this stock owned...
Stock: NVDA \ Action: [1]
Algorithm said 1, action changed to 2 to force sell out.
sell
Stock: AMD \ Action: [1]
Algorithm said 1, action changed to 2 to force sell out.
sell
Paper Trading Completed for Today!
